In [1]:
import gensim
from gensim import utils
import numpy as np
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import string
import glob
from tqdm import tqdm 
#import pdfminer
from pdfminer.high_level import extract_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
nltk.download('stopwords')
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
PAPER_OF_INTEREST_FNAME  = glob.glob('/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/*/*/*.pdf')

In [4]:
K = 0
print(PAPER_OF_INTEREST_FNAME[K])

/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/joss-papers-master/joss.00011/10.21105.joss.00011.pdf


In [5]:
Paper_interest = PAPER_OF_INTEREST_FNAME[K]

In [6]:
POI_PDF = [extract_text(Paper_interest)]
text = str(POI_PDF)

In [7]:
text

"['carl: a likelihood-free inference toolbox\\n\\nGilles Louppe1, Kyle Cranmer1, and Juan Pavez2\\n\\nDOI: 10.21105/joss.00011\\n\\n1 New York University 2 Federico Santa María University\\n\\nSummary\\n\\nSoftware\\n\\n• Review\\n• Repository\\n• Archive\\n\\nLicence\\nAuthors of JOSS papers retain\\ncopyright and release the work un-\\nder a Creative Commons Attri-\\nbution 4.0 International License\\n(CC-BY).\\n\\nCarl is a toolbox for likelihood-free inference in Python.\\n\\nThe likelihood function is the central object that summarizes the information from an\\nexperiment needed for inference of model parameters. It is key to many areas of science\\nthat report the results of classical hypothesis tests or confidence intervals using the (gen-\\neralized or profile) likelihood ratio as a test statistic. At the same time, with the advance\\nof computing technology, it has become increasingly common that a simulator (or gener-\\native model) is used to describe complex processes that 

In [ ]:
text = text.replace('-\\n','')
text = text.replace('-\n','')
text = text.replace('\\n',' ')
text = text.replace('\n', ' ')

In [ ]:
text

In [9]:
model = gensim.models.KeyedVectors.load_word2vec_format('/Volumes/Seagate Backup Plus Drive/JOSS Project/GoogleNews-vectors-negative300.bin', binary=True)

In [10]:
download('punkt') #tokenizer, run once
download('stopwords') #stopwords dictionary, run once
stop_words = stopwords.words('english')

def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
texts = (POI_PDF)

corpus = [preprocess(text) for text in texts]

In [12]:
corpus[0][0]

'carl'

In [13]:
def filter_docs(corpus, texts,  condition_on_doc):
    """
    Filter corpus, texts and labels given the function condition_on_doc which takes
    a doc.
    The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    #labels = [i for (i, doc) in zip(labels, corpus) if condition_on_doc(doc)]
    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [14]:
corpus, texts = filter_docs(corpus, texts,  lambda doc: (len(doc) != 0))

0 docs removed


In [15]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)

In [16]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

In [17]:
corpus, texts = filter_docs(corpus, texts,  lambda doc: has_vector_representation(model, doc))

0 docs removed


In [26]:
x =[]
for doc in corpus[0]: #look up each doc in model
    x.append(document_vector(model, doc))

In [27]:
X = np.array(x) #list to array

In [33]:
doc

'software'

In [44]:
np.unique(np.array(corpus[0]))

array(['abc', 'advance', 'aims', 'algorithms', 'along', 'alternative',
       'apparatus', 'approximate', 'approximating', 'archive', 'areas',
       'ative', 'authors', 'bayesian', 'become', 'bution', 'calibrated',
       'carl', 'carry', 'cases', 'central', 'classical', 'classifiers',
       'common', 'commons', 'companion', 'complex', 'computation',
       'computationally', 'computing', 'confidence', 'context',
       'copyright', 'cranmer', 'creative', 'density', 'der', 'describe',
       'details', 'development', 'directly', 'discriminative', 'doi',
       'eralized', 'estimation', 'et', 'evaluating', 'experiment',
       'federico', 'found', 'function', 'future', 'gilles', 'goal',
       'helpers', 'however', 'http', 'hypothesis', 'impossible',
       'impractical', 'including', 'increasingly', 'inference',
       'information', 'international', 'intervals', 'joss', 'journal',
       'juan', 'key', 'kyle', 'licence', 'license', 'likelihood',
       'louppe', 'many', 'march', 'ma

In [45]:
x2 =[]
for doc in np.unique(np.array(corpus[0])): #look up each doc in model
    x2.append(document_vector(model, doc))

In [46]:
X2 = np.array(x2) #list to array

In [47]:
average_word2vec_vector2 = X2.mean(axis=0)

In [48]:
model.similar_by_vector(average_word2vec_vector2)

[('h', 0.7580714225769043),
 ('o', 0.7527846097946167),
 ('r', 0.7336361408233643),
 ('f', 0.7306424379348755),
 ('b', 0.7227460145950317),
 ('ts', 0.7164600491523743),
 ('¬', 0.7156660556793213),
 ('ar', 0.7137598395347595),
 ('t', 0.6964871883392334),
 ('d', 0.6955010890960693)]

In [49]:
X2.shape

(132, 300)

In [28]:
X.shape

(173, 300)

In [29]:
X.mean(axis=0).shape

(300,)

In [31]:
average_word2vec_vector = X.mean(axis=0)

In [32]:
model.similar_by_vector(average_word2vec_vector)

[('h', 0.760456383228302),
 ('o', 0.7575953006744385),
 ('r', 0.7365941405296326),
 ('f', 0.7355577349662781),
 ('b', 0.7251432538032532),
 ('¬', 0.7178463339805603),
 ('ts', 0.7164381146430969),
 ('ar', 0.714815080165863),
 ('w', 0.6953476667404175),
 ('d', 0.6944249272346497)]

In [37]:
model.similar_by_vector(X[20])

[('h', 0.778066098690033),
 ('o', 0.7700537443161011),
 ('t', 0.7625486254692078),
 ('r', 0.7494496703147888),
 ('ts', 0.7428569197654724),
 ('u', 0.7180385589599609),
 ('¬', 0.7121788859367371),
 ('f', 0.7076941728591919),
 ('cks', 0.7059786319732666),
 ('ta', 0.698557436466217)]

In [35]:
model.most_similar('Journal', topn=20)

[('theJournal', 0.6163490414619446),
 ('theAmerican_Journal', 0.5941680669784546),
 ('Annals', 0.5925253629684448),
 ('Jounal', 0.5823094248771667),
 ('journal', 0.5741861462593079),
 ('journal_Annals', 0.5684102773666382),
 ('PATRICK_JACKSON', 0.5683403015136719),
 ('Physiology_Gastrointestinal', 0.5631081461906433),
 ('By_LULADEY_B.', 0.5580130219459534),
 ('Roentgenology_AJR', 0.557891845703125),
 ('Gazette', 0.5572773814201355),
 ('TADESSE', 0.5548046231269836),
 ('•_GANNETT_STAFF_WRITER', 0.5463052988052368),
 ('currier@sturgisjournal.com', 0.545045793056488),
 ('Herald', 0.5421846508979797),
 ('journal_Archives', 0.5398844480514526),
 ('journal_Cell_Metabolism', 0.5383172035217285),
 ('Bulletin', 0.5375578999519348),
 ('ESTEBAN_PARRA', 0.5362104177474976),
 ('Clinical_Nutrition_Vol', 0.5346276164054871)]